## 1. Planning Issues: Geared toward a functional and realistic goals

<img src="{{site.baseurl}}/images/travelplanner.png" height="400px">
<img src="{{site.baseurl}}/images/activityplanner.png" height="400px">



All great projects start off with solid planning. In my scrum team I took the planning initiative and helped consolidate the role of each of my team members as well as brainstorming how their features fit in to the overall project. I helped make insightful contributions to planning issues and documented our progress.

## 2. My Burndowns and Checklists (Kanban)

<img src="{{site.baseurl}}/images/kanban.png" height="350px">
<img src="{{site.baseurl}}/images/initialchecklist.png" height="275px">
<img src="{{site.baseurl}}/images/finalchecklist.png" height="350px">


I managed our teams kanban board making sure our tasks were following the AGILE methodology and making sure each person was making progress on their burndowns as scrum master. In addition I made use of a few checklists and burndowns to keep track of my progress and make sure my feature is complete.

## 3. Creating an API with Dynamic and Static Functionality + Table in Database

In [ ]:
import jwt
from flask import Flask, Blueprint, request, jsonify, g
from flask_restful import Api, Resource
from flask_cors import CORS
from datetime import datetime
from __init__ import app, db  # Ensure db is imported
from api.jwt_authorize import token_required
from model.post import Post
from model.rate import Rate
from flask_cors import cross_origin  # Importing cross_origin
from model.user import User


# Define the Blueprint for the Rate API
rate_api = Blueprint('rate_api', __name__, url_prefix='/api')
#CORS(rate_api, supports_credentials=True)
CORS(rate_api, supports_credentials=True, methods=['GET', 'POST', 'PUT', 'DELETE'])

# Connect the Api object to the Blueprint
api = Api(rate_api)

class RateAPI:
    """
    Define the API CRUD endpoints for the Rate model.
    There are operations for creating and retrieving ratings for a post.
    """

    class _CRUD(Resource):
        @token_required()
        @cross_origin(supports_credentials=True)  # Add this decorator to handle CORS for PUT requests
        def post(self):
            current_user = g.current_user
            data = request.get_json()
            post_id = data.get('post_id')
            rating_value = data.get('rating')

            if post_id is None or rating_value is None:
                return jsonify({"message": "post_id and rating are required"}), 400

            try:
                rating_value = int(rating_value)
            except ValueError:
                return jsonify({"message": "Rating must be an integer"}), 400

            if not (1 <= rating_value <= 10):
                return jsonify({"message": "Rating must be between 1 and 10"}), 400

            # Create a new rating entry
            rating = Rate(user_id=current_user.id, post_id=post_id, value=rating_value)
            db.session.add(rating)
            db.session.commit()

            return jsonify({"message": "Rating submitted successfully"})

        @token_required()
        def get(self):
            """
            Retrieve ratings for a post.
            """
            post_id = request.args.get('post_id')
            if not post_id:
                return jsonify({"message": "post_id is required"}), 400

            ratings = db.session.query(Rate, User).join(User, Rate.user_id == User.id).filter(Rate.post_id == post_id).all()
            ratings_list = [{"rating_id": r.Rate.id, "username": r.User._name, "rating": r.Rate.value} for r in ratings]

            return jsonify(ratings_list)

        
        @token_required()
        def put(self):
            """
            Update a rating (1-10 scale) for a post.
            """
            # Get current user from the token
            current_user = g.current_user
            # Get the request data
            data = request.get_json()
            rating_id = data.get('rating_id')
            new_rating_value = data.get('rating')

            # Validate rating_id and new_rating_value
            if rating_id is None or new_rating_value is None:
                return jsonify({"message": "rating_id and new rating value are required"}), 400

            try:
                new_rating_value = int(new_rating_value)
            except ValueError:
                return jsonify({"message": "Rating must be an integer"}), 400

            if not (1 <= new_rating_value <= 10):
                return jsonify({"message": "Rating must be between 1 and 10"}), 400

            # Find the rating by ID and join with User to get the user's role
            rating = db.session.query(Rate, User).join(User, Rate.user_id == User.id).filter(Rate.id == rating_id).first()

            if not rating:
                return jsonify({"message": "Rating not found"}), 404

            # Check if the current user is the owner of the rating or an admin
            if rating.Rate.user_id == current_user.id or current_user.role == 'Admin':
                # Update the rating value
                rating.Rate.value = new_rating_value
                db.session.commit()

                return jsonify({"message": "Rating updated successfully"})
            else:
                return jsonify({"message": "Not authorized to update this rating"}), 403
        
        @token_required()
        def delete(self):
            """
            Delete a rating for a post by the current user.
            """
            current_user = g.current_user
            data = request.get_json()
            rating_id = data.get('rating_id')
    
            if rating_id is None:
                return jsonify({"message": "rating_id is required"}), 400
            
            rating = db.session.query(Rate, User).join(User, Rate.user_id == User.id).filter(Rate.id == rating_id).first()
            
            if rating.Rate.user_id == current_user.id or current_user.role == 'Admin':
                # Ensure the rating belongs to the current user
                rating = Rate.query.filter_by(id=rating_id).first()
                if rating:
                    db.session.delete(rating)
                    db.session.commit()
                    return jsonify({"message": "Rating deleted successfully"})
                else:
                    return jsonify({"message": "Rating not found"}), 404
           

# Add resource to the API
api.add_resource(RateAPI._CRUD, "/rate")

In [ ]:
rates = [
            Rate(value=5, user_id=5, post_id=1),
            Rate(value=8, user_id=6, post_id=1),
            Rate(value=1, user_id=7, post_id=1),
            Rate(value=7, user_id=8, post_id=1),
            Rate(value=1, user_id=9, post_id=1),

            Rate(value=8, user_id=10, post_id=2),
            Rate(value=6, user_id=11, post_id=2),
            Rate(value=3, user_id=12, post_id=2),
            Rate(value=8, user_id=13, post_id=2),
            Rate(value=10, user_id=14, post_id=2),

            Rate(value=8, user_id=15, post_id=3),
            Rate(value=7, user_id=16, post_id=3),
            Rate(value=6, user_id=17, post_id=3),
            Rate(value=5, user_id=18, post_id=3),
            Rate(value=5, user_id=19, post_id=3),

            Rate(value=8, user_id=20, post_id=4),
            Rate(value=4, user_id=21, post_id=4),
            Rate(value=3, user_id=22, post_id=4),
            Rate(value=8, user_id=23, post_id=4),
            Rate(value=2, user_id=24, post_id=4),

            Rate(value=2, user_id=25, post_id=5),
            Rate(value=2, user_id=26, post_id=5),
            Rate(value=1, user_id=27, post_id=5),
            Rate(value=5, user_id=28, post_id=5)
        ]


## 4. Created a Silent CPT Demo for AP Exam

[Watch the video](https://youtu.be/jNFIHRm9wgk)

<iframe width="600" height="400" src="https://youtu.be/jNFIHRm9wgk" frameborder="0" allowfullscreen></iframe>

Through my creation of the silent CPT video I made sure to demo:
- The use/functionality of my Gemini chatbot to ask travel questions
- My card page with average ratings and different attractions
- A live rating bar
- A table with different ratings, and demonstrated CRUD functions and live rating changes

## 5. Holding stand-ups and maintaining a coherent focus toward the project of Intertravel across my entire group's features

<img src="{{site.baseurl}}/images/standup1.png" height="200px">
<img src="{{site.baseurl}}/images/standup2.png" height="200px">

By organizing frequent stand-ups with my team, at both the whitebaord and my table I was able to:
 - Convey the requirements and what we should be working on
 - Feedback for specific teammates
 - Areas of focus
 - Align the team, and make sure our individual features were unique and had valuable contributions to the overall InterTravel project

## 6. Organized a Live Review with Ms. Pataki for my team

<img src="{{site.baseurl}}/images/mspatakireview.png" height="200px">

We gained valuable insight through the live review with Ms. Pataki
 - We learned how to pace ourselves
 - Learned to highlight the most important and attractive features
 - Gained the ability to explain features and their real-world application
 - Gained confidence and practice to present to larger groups